In [3]:
import os
import pandas as pd
from docx import Document
import re
import openpyxl
from openpyxl.styles import PatternFill,Font, Alignment


# Function to extract tables from a DOCX file and convert them into DataFrames
def extract_tables_from_doc(doc):
    """Extract tables from a DOCX document and convert them into a list of DataFrames."""
    tables = []
    for table in doc.tables:
        rows = [[cell.text.strip() for cell in row.cells] for row in table.rows]
        if rows:
            df = pd.DataFrame(rows[1:], columns=rows[0])  # Use the first row as headers
            tables.append(df)
    return [df for df in tables if not df.empty]

# Function to normalize strings for comparison
def normalize_string(s, normalization_map):
    """Normalize the string based on patterns."""
    s = s.lower()
    s = re.sub(r'(january|february|march|april|may|june|july|august|september|october|november|december)\s*\d{1,2},\s*\d{4}', '', s)
    s = s.replace("(loss)", "").replace("(income)", "")
    for key, value in normalization_map.items():
        if key in s:
            s = value
    return s.replace(" ", "")

# Function to calculate match percentage between two DataFrames
def match_percentage(df1, df2, normalization_map):
    """Calculate the match percentage between two DataFrames based on their first column."""
    list1 = [normalize_string(row[0], normalization_map) for row in df1.iloc[:, [0]].values]
    list2 = [normalize_string(row[0], normalization_map) for row in df2.iloc[:, [0]].values]

    set1 = set(list1)
    set2 = set(list2)

    intersection = set1.intersection(set2)
    match_pct = (len(intersection) / min(len(set1), len(set2))) * 100 if len(set1) > 0 else 0
    return match_pct

# Function to create side-by-side DataFrames for comparison
def create_side_by_side_df(df1, df2):
    """Combine two DataFrames side by side for visual comparison."""
    def make_columns_unique(df):
        """Append a numeric suffix to duplicate column names."""
        seen = {}
        new_columns = []
        for col in df.columns:
            if col in seen:
                seen[col] += 1
                new_columns.append(f"{col}_{seen[col]}")
            else:
                seen[col] = 0
                new_columns.append(col)
        df.columns = new_columns
        return df

    # Work on copies to avoid modifying the original DataFrames
    df1 = make_columns_unique(df1.copy())
    df2 = make_columns_unique(df2.copy())

    # Resize both DataFrames to align their rows
    max_rows = max(len(df1), len(df2))
    df1_resized = df1.reindex(range(max_rows), fill_value='').reset_index(drop=True)
    df2_resized = df2.reindex(range(max_rows), fill_value='').reset_index(drop=True)

    # Add suffixes for clarity
    df1_resized.columns = [f"_Original{col}" for col in df1_resized.columns]
    df2_resized.columns = [f"_Target{col}" for col in df2_resized.columns]

    # Concatenate side by side
    return pd.concat([df2_resized, df1_resized], axis=1)



def highlight_differences_colour(workbook):
    # Iterate through all sheets with the prefix 'Differences_'
    for sheet_name in workbook.sheetnames:
        if sheet_name.startswith("Differences_"):
            sheet = workbook[sheet_name]

            # Identify the column indexes for "_Original" and "_Target"
            original_column_idx = None
            target_column_idx = None

            for col_idx, col in enumerate(sheet.columns, 1):
                col_name = str(col[0].value).strip().lower() if col[0].value else ""
                if "_original" in col_name:
                    original_column_idx = col_idx
                    break  # Exit loop once the column is found

            # Find the index for the "_Target" column
            for col_idx, col in enumerate(sheet.columns, 1):
                col_name = str(col[0].value).strip().lower() if col[0].value else ""
                if "_target" in col_name:
                    target_column_idx = col_idx
                    break  # Exit loop once the column is found

            # If both columns are found, proceed with comparison
            if original_column_idx and target_column_idx:
                # Get all values in the '_Original' column and '_Target' column
                original_values = [sheet.cell(row=row, column=original_column_idx).value for row in range(2, sheet.max_row + 1)]
                target_values = [sheet.cell(row=row, column=target_column_idx).value for row in range(2, sheet.max_row + 1)]

                # Create sets of original and target values for comparison
                original_set = set(original_values)
                target_set = set(target_values)

                # Find differences
                missing_in_target = original_set - target_set
                extra_in_target = target_set - original_set

                # Add a column for the colored flag at the end
                colored_flag_column_idx = sheet.max_column + 1
                sheet.cell(row=1, column=colored_flag_column_idx).value = "Line Item Diff. Flag"
                sheet.cell(row=1, column=colored_flag_column_idx).font = Font(bold=True)

                # Iterate through rows to apply coloring and set the flag
                for row in range(2, sheet.max_row + 1):
                    is_colored = False  # Flag to track if any cell is colored in the row

                    # Get the key value from the first column and convert it to lowercase
                    key_value = str(sheet.cell(row=row, column=1).value).strip().lower() if sheet.cell(row=row, column=1).value else ""

                    # Skip highlighting if the key value is '_target' or '_original'
                    if key_value in ["_target", "_original"]:
                        continue

                    # Highlight missing values in '_Original' column (Red)
                    original_value = sheet.cell(row=row, column=original_column_idx).value
                    if original_value in missing_in_target:
                        sheet.cell(row=row, column=original_column_idx).fill = PatternFill(start_color="FFA07A", end_color="FFA07A", fill_type="solid")
                        is_colored = True  # Set flag to True if colored

                    # Highlight extra values in '_Target' column (Green)
                    target_value = sheet.cell(row=row, column=target_column_idx).value
                    if target_value in extra_in_target:
                        sheet.cell(row=row, column=target_column_idx).fill = PatternFill(start_color="FFB347", end_color="FFB347", fill_type="solid")
                        is_colored = True  # Set flag to True if colored

                    # If any cell in the row is colored, set the colored flag column to 1
                    if is_colored:
                        sheet.cell(row=row, column=colored_flag_column_idx).value = 1
                    else:
                        sheet.cell(row=row, column=colored_flag_column_idx).value = 0

# Paths for input and output files
input_folder = "Input Files"
output_folder = "Output Files"
# input_folder = "./"
# output_folder = "./"
original_filename= "Q1FY25 Quarterly Report.docx"
target_filename =  "Q2FY25 Quarterly Report.docx"
docx_file_1 = os.path.join(input_folder, original_filename )
docx_file_2 = os.path.join(input_folder, target_filename)
output_file = docx_file_2.replace('Input Files/', '').replace('.docx', '')

# Load DOCX files and extract tables
doc1 = Document(docx_file_1)
doc2 = Document(docx_file_2)
tables_doc1 = extract_tables_from_doc(doc1)
tables_doc2 = extract_tables_from_doc(doc2)

# Normalization rules
normalization_map = {
    "balance as of": "balance as of",
    "balance at": "balance at",
    "realized gains": "realized gains",
    "unrealized gains": "realized gains",
    "realized losses": "realized losses",
    "unrealized losses": "realized losses",
    "net loss": "net loss",
    "net income": "net loss"
}

# Document mapping for comparison
doc1_dict = {
    "Balance Sheet": [tables_doc1[3]],
    "Income Statement": [tables_doc1[4], tables_doc1[5], tables_doc1[6], tables_doc1[7]],
    "Statement Of Equity": [tables_doc1[8], tables_doc1[9], tables_doc1[8], tables_doc1[9]],
    "Cash Flow": [tables_doc1[10]],
    "Fair Value Measurements - Current ": [tables_doc1[11]],
    "Fair Value Measurements - Past ": [tables_doc1[12]],
    "Marketable Securities - Current": [tables_doc1[13]],
    "Marketable Securities - Past": [tables_doc1[14]],
    "Marketable Securities - Dues": [tables_doc1[15]],
    "Car Val Pvt Debt Securities": [tables_doc1[16], tables_doc1[17]],
    "Car Val Pvt Equity Securities": [tables_doc1[18], tables_doc1[19]],
    "Gains and Losses on Srtg Inv": [tables_doc1[20]],
    "Equity Method Investment": [tables_doc1[21]],
    "Nat. Amt of Derivative Instr": [tables_doc1[22], tables_doc1[23]],
    "Fair Value Derivatives": [tables_doc1[24]],
    "Pre-Tax Effects Derivatives": [tables_doc1[25]],
    "Property and Equipment": [tables_doc1[26]],
    "Goodwill": [tables_doc1[27]],
    "Intangible Assets": [tables_doc1[28]],
    "Future Amortization Expenses": [tables_doc1[29]],
    "Accrued Exp and Oth Crr Liabl": [tables_doc1[30]],
    "Senior Notes": [tables_doc1[31]],
    "Operating Leases": [tables_doc1[32]],
    "Disagg Revenue by Geo": [tables_doc1[33]],
    "Disagg Revenue by Deployment": [tables_doc1[34]],
    "Deferred Revenue": [tables_doc1[35]],
    "Deferred Contct Acquisition Cos": [tables_doc1[36]],
    "Stockholder's Equity": [tables_doc1[37]],
    "Net loss per Share": [tables_doc1[38]],
    "Weighted average dilutive securities": [tables_doc1[39]]
}

# Threshold and storage for results
target_match_threshold = 65
matched_tables = {}
differences_dict = {}
index_data = []
matched_doc2_indices = set()

# Perform matching and generate differences
for target_df_name, target_dfs in doc1_dict.items():
    matched_tables[target_df_name] = []
    differences_dict[target_df_name] = []
    max_match_pct = 0
    is_present = "No"

    for target_df in target_dfs:
        for i, df in enumerate(tables_doc2):
            if i in matched_doc2_indices:
                continue

            match_pct = match_percentage(target_df, df, normalization_map)
            max_match_pct = max(max_match_pct, match_pct)

            if match_pct >= target_match_threshold:
                matched_tables[target_df_name].append(df.copy())
                matched_doc2_indices.add(i)

                if match_pct < 100:
                    clean_target_df = target_df.copy()
                    clean_df = df.copy()
                    side_by_side_df = create_side_by_side_df(clean_target_df, clean_df)
                    differences_dict[target_df_name].append(side_by_side_df)
                break

    if max_match_pct >= target_match_threshold:
        is_present = "Yes"

    index_data.append([target_df_name, max_match_pct, is_present])

# Create the output Excel file
output_excel_file = os.path.join(output_folder, output_file + '_Recentv2.xlsx')
with pd.ExcelWriter(output_excel_file, engine="openpyxl") as writer:
    index_data_with_validation = []
    for data in index_data:
        table_name, match_pct, is_present = data
        if match_pct < 100:
            validation_required = f"Validation Required! Please refer to Differences_{table_name}"
        else:
            validation_required = "Not Required"
        index_data_with_validation.append(data + [validation_required])

  
    index_df = pd.DataFrame(index_data_with_validation, columns=["Financial Document Name", "Match %", "Extract Successful?", "Validation Required?"])
    #index_df=index_df[["Financial Document Name",  "Extract Successful?", "Validation Required?"]]
    index_df.to_excel(writer, sheet_name="Index", index=False)

    for target_df_name, tables in matched_tables.items():
        start_row = 0
        for table in tables:
            table.to_excel(writer, sheet_name=target_df_name, index=False, startrow=start_row)
            start_row += len(table) + 2

    for target_df_name, side_by_side_dfs in differences_dict.items():
        start_row = 0
        for side_by_side_df in side_by_side_dfs:
            side_by_side_df.to_excel(writer, sheet_name=f"Differences_{target_df_name}", index=False, startrow=start_row)
            start_row += len(side_by_side_df) + 2

    workbook = writer.book




# Function to replace continuous repeated horizontal values in the first column with '-'
def process_original_sheets_continuous(workbook):
    """Replace continuous repeated horizontal values in the first column of original sheets with '-'."""
    for sheet_name in workbook.sheetnames:
        if not sheet_name.startswith("Differences_"):  # Only process original sheets
            sheet = workbook[sheet_name]

            # Loop through all rows in the sheet
            for row in range(2, sheet.max_row + 1):  # Skip the header row
                first_col_value = sheet.cell(row=row, column=1).value  # Value in the first column
                if first_col_value:  # Check if the first column value exists
                    # Start tracking the last seen value
                    last_value = first_col_value

                    # Loop through subsequent columns in the same row
                    for col in range(2, sheet.max_column + 1):
                        current_value = sheet.cell(row=row, column=col).value

                        # Replace with '-' only if the value matches and is continuous
                        if current_value == last_value:
                            sheet.cell(row=row, column=col).value = ' '
                        else:
                            # Break if a non-continuous value is found
                            last_value = current_value

def add_labels_to_difference_sheets(workbook, target_filename, original_filename):
    """
    Replace 'Original_' and 'Target_' prefixes in headers with formatted document names.
    """
    for sheet_name in workbook.sheetnames:
        if sheet_name.startswith("Differences_"):
            sheet = workbook[sheet_name]

            # Get the total number of columns
            max_col = sheet.max_column

            # Iterate through each column to update headers
            for col in range(1, max_col + 1):
                header_value = sheet.cell(row=1, column=col).value
                if header_value:
                    if "_Target" in header_value:
                        # Replace '_Target' with formatted target filename
                        new_header = header_value.replace("_Target", f"Current 10Q: {target_filename} : ")
                        sheet.cell(row=1, column=col).value = new_header
                        sheet.cell(row=1, column=col).font = Font(bold=True, size=14)
                        sheet.cell(row=1, column=col).alignment = Alignment(horizontal="center", vertical="center")

                    if "_Original" in header_value:
                        # Replace '_Original' with formatted original filename
                        new_header = header_value.replace("_Original", f"Blue Print 10Q: {original_filename} : ")
                        sheet.cell(row=1, column=col).value = new_header
                        sheet.cell(row=1, column=col).font = Font(bold=True, size=14)
                        sheet.cell(row=1, column=col).alignment = Alignment(horizontal="center", vertical="center")

    # Save the workbook after modifications
    workbook.save(output_excel_file)





# def add_labels_to_difference_sheets(workbook, target_filename, original_filename):
#     """
#     Add labels 'Target' and 'Original' in the first row of 'Differences_' sheets.
#     Push the data ddd_lown to start from the second row.
#     """
#     for sheet_name in workbook.sheetnames:
#         if sheet_name.startswith("Differences_"):
#             sheet = workbook[sheet_name]

#             # Insert a new row at the top to push data down
#             sheet.insert_rows(1)

#             # Get the total number of columns
#             max_col = sheet.max_column

#             # Add 'Target' label above the "_Target" columns
#             for col in range(1, max_col + 1):
#                 header_value = sheet.cell(row=2, column=col).value
#                 if "_Target" in str(header_value):
#                     sheet.cell(row=1, column=col).value = f"Current 10Q: {target_filename}"
#                     sheet.cell(row=1, column=col).font = Font(bold=True, size=14)
#                     sheet.cell(row=1, column=col).alignment = Alignment(horizontal="center", vertical="center")

#                 # Add 'Original' label above the "_Original" columns
#                 if "_Original" in str(header_value):
#                     sheet.cell(row=1, column=col).value = f"Blue Print 10Q: {original_filename}"
#                     sheet.cell(row=1, column=col).font = Font(bold=True, size=14)
#                     sheet.cell(row=1, column=col).alignment = Alignment(horizontal="center", vertical="center")

#     # Save the workbook after modifications
#     workbook.save(output_excel_file)



highlight_differences_colour(workbook)
workbook.save(output_excel_file)
# Apply processing to original sheets
process_original_sheets_continuous(workbook)

# Call the function with the necessary inputs
add_labels_to_difference_sheets(workbook, target_filename, original_filename)
workbook.save(output_excel_file)





/Users/dhanunjaikumar/Library/Python/3.9/lib/python/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
